<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Train-test Split and Cross-Validation Lab

_Authors: Joseph Nelson (DC), Kiefer Katovich (SF)_

---

## Review of train/test validation methods

We've discussed overfitting, underfitting, and how to validate the "generalizeability" of your models by testing them on unseen data. 

In this lab you'll practice two related validation methods: 
1. **train/test split**
2. **k-fold cross-validation**

Train/test split and k-fold cross-validation both serve two useful purposes:
- We prevent overfitting by not using all the data, and
- We retain some remaining data to evaluate our model.

In the case of cross-validation, the model fitting and evaluation is performed multiple times on different train/test splits of the data.

Ultimately we can the training and testing validation framework to compare multiple models on the same dataset. This could be comparisons of two linear models, or of completely different models on the same data.


## Instructions

For your independent practice, fit **three different models** on the Boston housing data. For example, you could pick three different subsets of variables, one or more polynomial models, or any other model that you like. 

**Start with train/test split validation:**
* Fix a testing/training split of the data
* Train each of your models on the training data
* Evaluate each of the models on the test data
* Rank the models by how well they score on the testing data set.

**Then try K-Fold cross-validation:**
* Perform a k-fold cross validation and use the cross-validation scores to compare your models. Did this change your rankings?
* Try a few different K-splits of the data for the same models.

If you're interested, try a variety of response variables.  We start with **MEDV** (the `.target` attribute from the dataset load method).

In [1]:
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston

boston = load_boston()

In [9]:
# A:
boston

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]),
 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
        15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
        13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
        21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
        35.4, 24.7, 3

In [12]:
# Create a DataFrame for both parts of data; don't forget to assign column names.
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=['MEDV'])
boston = pd.concat([y, X], axis=1)

### 1. Clean up any data problems

Load the Boston housing data.  Fix any problems, if applicable.

In [13]:
# A: 
boston.describe()

,MEDV,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,22.532806,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,9.197104,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,5.000000,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,17.025000,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,21.200000,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,25.000000,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,50.000000,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


Look at the documentation here for how to split data into different sizes: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

### 2. Select 3-4 variables with your dataset to perform a 50/50 test train split on

- Use sklearn.
- Score and plot your predictions.

In [22]:
# A:
# create feature matrix (X)
feature_cols = boston.columns.drop(['MEDV'])
X = boston[feature_cols]

# create response vector (y)
y = boston.MEDV

# Create a DataFrame for both parts of data; don't forget to assign column names.
boston = pd.concat([y, X], axis=1)

from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5)

# Before splitting
print(X.shape)
print(y.shape)

# After splitting
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

# WITHOUT a random_state parameter:
#  (If you run this code several times, you get different results!)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Print the first element of each object.
print(X_train.head(1))

# WITH a random_state parameter:
#  (Same split every time! Note you can change the random state to any integer.)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Print the first element of each object.
print(X_train.head(1))
print(X_test.head(1))
print(y_train.head(1))
print(y_test.head(1))

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

(506, 13)
(506,)
(253, 13)
(253, 13)
(253,)
(253,)
        CRIM   ZN  INDUS  CHAS    NOX     RM   AGE    DIS  RAD    TAX  \
221  0.40771  0.0    6.2   1.0  0.507  6.164  91.3  3.048  8.0  307.0   

     PTRATIO       B  LSTAT  
221     17.4  395.24  21.46  
        CRIM   ZN  INDUS  CHAS    NOX    RM   AGE     DIS  RAD    TAX  \
502  0.04527  0.0  11.93   0.0  0.573  6.12  76.7  2.2875  1.0  273.0   

     PTRATIO      B  LSTAT  
502     21.0  396.9   9.08  
        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
307  0.04932  33.0   2.18   0.0  0.472  6.849  70.3  3.1827  7.0  222.0   

     PTRATIO      B  LSTAT  
307     18.4  396.9   7.53  
502    20.6
Name: MEDV, dtype: float64
307    28.2
Name: MEDV, dtype: float64


LinearRegression()

In [23]:
y_pred = lr.predict(X_test)

print(np.sqrt(metrics.mean_squared_error(y_train, lr.predict(X_train))))
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

4.74109521333182
4.679504823808762


### 3. Try 70/30 and 90/10
- Score and plot.  
- How do your metrics change?

In [25]:
# A:
# A:
# create feature matrix (X)
feature_cols = boston.columns.drop(['MEDV'])
X = boston[feature_cols]

# create response vector (y)
y = boston.MEDV

# Create a DataFrame for both parts of data; don't forget to assign column names.
boston = pd.concat([y, X], axis=1)

from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.3)

# Before splitting
print(X.shape)
print(y.shape)

# After splitting
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

# WITHOUT a random_state parameter:
#  (If you run this code several times, you get different results!)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Print the first element of each object.
print(X_train.head(1))

# WITH a random_state parameter:
#  (Same split every time! Note you can change the random state to any integer.)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Print the first element of each object.
print(X_train.head(1))
print(X_test.head(1))
print(y_train.head(1))
print(y_test.head(1))

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print(np.sqrt(metrics.mean_squared_error(y_train, lr.predict(X_train))))
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

(506, 13)
(506,)
(354, 13)
(152, 13)
(354,)
(152,)
        CRIM   ZN  INDUS  CHAS   NOX     RM    AGE     DIS   RAD    TAX  \
437  15.1772  0.0   18.1   0.0  0.74  6.152  100.0  1.9142  24.0  666.0   

     PTRATIO     B  LSTAT  
437     20.2  9.32  26.45  
        CRIM   ZN  INDUS  CHAS    NOX    RM   AGE     DIS  RAD    TAX  \
502  0.04527  0.0  11.93   0.0  0.573  6.12  76.7  2.2875  1.0  273.0   

     PTRATIO      B  LSTAT  
502     21.0  396.9   9.08  
        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
307  0.04932  33.0   2.18   0.0  0.472  6.849  70.3  3.1827  7.0  222.0   

     PTRATIO      B  LSTAT  
307     18.4  396.9   7.53  
502    20.6
Name: MEDV, dtype: float64
307    28.2
Name: MEDV, dtype: float64
4.74109521333182
4.679504823808762


In [26]:
# create feature matrix (X)
feature_cols = boston.columns.drop(['MEDV'])
X = boston[feature_cols]

# create response vector (y)
y = boston.MEDV

# Create a DataFrame for both parts of data; don't forget to assign column names.
boston = pd.concat([y, X], axis=1)

from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.1)

# Before splitting
print(X.shape)
print(y.shape)

# After splitting
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

# WITHOUT a random_state parameter:
#  (If you run this code several times, you get different results!)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Print the first element of each object.
print(X_train.head(1))

# WITH a random_state parameter:
#  (Same split every time! Note you can change the random state to any integer.)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Print the first element of each object.
print(X_train.head(1))
print(X_test.head(1))
print(y_train.head(1))
print(y_test.head(1))

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print(np.sqrt(metrics.mean_squared_error(y_train, lr.predict(X_train))))
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

(506, 13)
(506,)
(455, 13)
(51, 13)
(455,)
(51,)
        CRIM   ZN  INDUS  CHAS    NOX     RM   AGE     DIS   RAD    TAX  \
459  6.80117  0.0   18.1   0.0  0.713  6.081  84.4  2.7175  24.0  666.0   

     PTRATIO      B  LSTAT  
459     20.2  396.9   14.7  
        CRIM   ZN  INDUS  CHAS    NOX    RM   AGE     DIS  RAD    TAX  \
502  0.04527  0.0  11.93   0.0  0.573  6.12  76.7  2.2875  1.0  273.0   

     PTRATIO      B  LSTAT  
502     21.0  396.9   9.08  
        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
307  0.04932  33.0   2.18   0.0  0.472  6.849  70.3  3.1827  7.0  222.0   

     PTRATIO      B  LSTAT  
307     18.4  396.9   7.53  
502    20.6
Name: MEDV, dtype: float64
307    28.2
Name: MEDV, dtype: float64
4.74109521333182
4.679504823808762


### 4. Try K-Folds cross-validation with K between 5-10 for your regression. 

- What seems optimal? 
- How do your scores change?  
- What the variance of scores like?
- Try different folds to get a sense of how this impacts your score.

In [28]:
# A:
from sklearn import model_selection
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=22)

for train_index, test_index in kf.split(X, y):
    print(len(train_index), train_index)
    print(len(test_index), test_index)

404 [  0   1   4   5   6   7   8   9  10  11  12  14  15  16  17  18  19  20
  21  23  24  25  26  27  29  30  32  33  34  36  37  38  39  41  43  44
  45  47  48  49  50  51  52  53  54  55  56  57  59  60  61  62  63  64
  65  68  69  70  72  74  75  76  78  79  80  81  82  83  84  85  86  87
  88  90  91  92  93  94  95  96  97  98  99 100 103 104 105 106 107 108
 109 110 112 113 114 115 116 117 118 120 121 124 125 126 128 129 131 132
 133 134 135 136 138 139 140 141 142 143 144 145 146 147 149 150 151 153
 154 155 156 158 159 160 161 162 164 165 167 168 169 170 171 172 175 176
 177 179 181 182 183 184 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 203 204 205 206 207 210 211 212 213 214 215 216 217 218
 220 223 224 225 226 227 228 229 230 231 232 233 234 237 238 239 240 241
 242 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 265 267 268 269 270 272 273 274 275 276 277 278 279 281 283 285 286 287
 288 289 290 291 293 294 295 296 297 298 299 30

In [29]:
mse_values = []
scores = []
n = 0

print("~~~~ CROSS VALIDATION each fold ~~~~")
for train_index, test_index in kf.split(X, y):
    
    lr = LinearRegression().fit(X.iloc[train_index], y.iloc[train_index])
    
    mse_values.append(np.sqrt(metrics.mean_squared_error(y.iloc[test_index], lr.predict(X.iloc[test_index]))))
    scores.append(lr.score(X, y))
    
    n += 1
    
    print('Model {}'.format(n))
    print('RMSE: {}'.format(mse_values[n-1]))
    print('R2: {}\n'.format(scores[n-1]))


print("~~~~ SUMMARY OF CROSS VALIDATION ~~~~")
print('Mean of RMSE for all folds: {}'.format(np.mean(mse_values)))
print('Mean of R2 for all folds: {}'.format(np.mean(scores)))

~~~~ CROSS VALIDATION each fold ~~~~
Model 1
RMSE: 4.557486674063903
R2: 0.7372764083054519

Model 2
RMSE: 4.947676432287829
R2: 0.7384075559041092

Model 3
RMSE: 5.040685667588866
R2: 0.7380846085724404

Model 4
RMSE: 5.449415076048826
R2: 0.7379411577652356

Model 5
RMSE: 4.215218147998681
R2: 0.7399617090488524

~~~~ SUMMARY OF CROSS VALIDATION ~~~~
Mean of RMSE for all folds: 4.8420963995976205
Mean of R2 for all folds: 0.738334287919218


In [34]:
from sklearn.model_selection import cross_val_score

# Note the results will vary each run since we take a different
#   subset of the data each time (since shuffle=True)
kf = model_selection.KFold(n_splits=10, shuffle=True, random_state=22)

print("CV Root Mean Squared Error: ", np.mean(-cross_val_score(lr, X, y, cv=kf, scoring='neg_root_mean_squared_error')))
print("CV R2 Score: ", np.mean(cross_val_score(lr, X, y, cv=kf)))

CV Root Mean Squared Error:  4.814615185373965
CV R2 Score:  0.7043215533554819


In [35]:
# A:
from sklearn import model_selection
kf = model_selection.KFold(n_splits=7, shuffle=True, random_state=22)

for train_index, test_index in kf.split(X, y):
    print(len(train_index), train_index)
    print(len(test_index), test_index)

433 [  0   1   4   5   6   7   8   9  10  11  12  14  15  16  17  18  19  20
  21  23  24  25  26  27  29  30  32  33  34  36  37  38  39  41  42  43
  44  45  46  47  48  49  50  51  52  53  54  55  56  57  59  60  61  62
  63  64  65  66  67  68  69  70  72  74  75  76  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 120
 121 122 124 125 126 128 129 131 132 133 134 135 136 138 139 140 141 142
 143 144 145 146 147 149 150 151 153 154 155 156 158 159 160 161 162 164
 165 167 168 169 170 171 172 173 175 176 177 179 181 182 183 184 185 186
 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 203 204 205
 206 207 208 210 211 212 213 214 215 216 217 218 220 221 223 224 225 226
 227 228 229 230 231 232 233 234 237 238 239 240 241 242 244 245 246 247
 248 249 250 251 252 253 254 255 256 257 258 259 260 261 265 267 268 269
 270 272 273 274 275 276 277 278 279 281 283 28

In [36]:
mse_values = []
scores = []
n = 0

print("~~~~ CROSS VALIDATION each fold ~~~~")
for train_index, test_index in kf.split(X, y):
    
    lr = LinearRegression().fit(X.iloc[train_index], y.iloc[train_index])
    
    mse_values.append(np.sqrt(metrics.mean_squared_error(y.iloc[test_index], lr.predict(X.iloc[test_index]))))
    scores.append(lr.score(X, y))
    
    n += 1
    
    print('Model {}'.format(n))
    print('RMSE: {}'.format(mse_values[n-1]))
    print('R2: {}\n'.format(scores[n-1]))


print("~~~~ SUMMARY OF CROSS VALIDATION ~~~~")
print('Mean of RMSE for all folds: {}'.format(np.mean(mse_values)))
print('Mean of R2 for all folds: {}'.format(np.mean(scores)))

~~~~ CROSS VALIDATION each fold ~~~~
Model 1
RMSE: 4.742001239157448
R2: 0.7385214365228067

Model 2
RMSE: 4.9869624229283405
R2: 0.7393189692554034

Model 3
RMSE: 4.0685177934292165
R2: 0.7392285114690653

Model 4
RMSE: 5.654665175562927
R2: 0.7385206745481128

Model 5
RMSE: 5.183314046718985
R2: 0.7393400566215976

Model 6
RMSE: 5.140721532967211
R2: 0.7386879717467254

Model 7
RMSE: 4.025565793590211
R2: 0.7396644958453318

~~~~ SUMMARY OF CROSS VALIDATION ~~~~
Mean of RMSE for all folds: 4.828821143479191
Mean of R2 for all folds: 0.7390403022870061


### 5. [Bonus] optimize the $R^2$ score

Can you optimize your R^2 by selecting the best features and validating the model using either train/test split or K-Folds?

Your code will need to iterate through the different combinations of predictors, cross-validate the current model parameterization, and determine which set of features performed best.

The number of K-folds is up to you.

> *Hint:* the `itertools` package is useful for combinations and permutations.


In [ ]:
# A:

### 5.1 Can you explain what could be wrong with this approach?

In [ ]:
# A:

### 6. [Bonus] Explore another target variable

Can you find another response variable, given a combination of predictors, that can be predicted accurately through the exploration of different predictors in this dataset?



> *Tip: Check out pairplots, coefficients, and pearson scores.*

In [ ]:


# A: